In [2]:
import numpy as np
import scipy.linalg as sp

In [3]:
from numpy import pi, cos
from numpy.linalg import cond, det
from scipy.linalg import norm, svd

n = 12
A = np.ones((n+1,n+1))
for i in range(1, n+1):
    for j in range(1, n+1):
        A[i,j] = 1/((2*i)**j)
sumA = A.sum()
print(f"sumA: {sumA}")

condA = cond(A, 2)
print(f"condA: {condA}")

comment1 = 'no'
print(f"comment1: {comment1}")

detA = det(A)
print(f"detA: {detA}")

comment2 = 'no' if abs(detA) < np.finfo(float).eps else 'yes'
print(f"comment2: {comment2}")

_, svA, _ = svd(A)
SumsvA = svA.sum()
print(f"SumsvA: {SumsvA}")


V = A[:6, :6]
xpts1 = np.array([1, 1/2, 1/4, 1/6, 1/8, 1/10])
fx1 = lambda x : (x+1) * cos(2*pi*x)
y1 = fx1(xpts1)
veca = sp.solve(V, y1)
print(f"veca: {veca}")

sumA: 27.224108678060873
condA: 3.161309735587093e+17
comment1: no
detA: -1.2411566175315346e-80
comment2: no
SumsvA: 7.520671865894069
veca: [  1.00798809   0.7310131  -16.13915809 -44.18283957 153.30979796
 -92.72680149]


In [4]:
from numpy import zeros, exp
from scipy.optimize import fsolve

f2_1 = lambda x : (x-1)**4
f2_2 = lambda x : 3 - exp(x)
g2 = lambda x : f2_1(x) - f2_2(x)
dg2 = lambda x : 4*(x-1)**3 + exp(x)
newton_step = lambda x : x - g2(x)/dg2(x)

x0 = 5/2
x1 = newton_step(x0)
x2 = newton_step(x1)
x3 = newton_step(x2)
print(f"x1: {x1}, x2: {x2}, x3: {x3}")

tol = 1e-8
xn = x0
xe = None
while True:
    xe = newton_step(xn)
    if (abs(xe - xn)) < tol:
        break
    xn = xe
print(f"xe: {xe}")
f1f2 = xe
print(f"f1f2: {f1f2}")

x1: 1.9453422637809399, x2: 1.4832195389998697, x3: 1.1823751896755503
xe: 1.0985808071939192
f1f2: 1.0985808071939192


In [5]:
from numpy import ones, sqrt, pi, cos, exp, linspace
from scipy.special import erf

N = 10
h = 1/N
x = linspace(0, 1, N+1)
w = np.ones(N+1)
w[1::2] = 4
w[2:-1:2] = 2
f3 = lambda x : exp(x) * sqrt(1-x)
f = f3(x)
simpf = h/3 * w.T @ f
print(f"simpf: {simpf}")

true = sqrt(pi) * erf(1) * exp(1) / 2 - 1
errsimp = abs(true - simpf)
print(f"errsimp: {errsimp}")

F3 = lambda x,y : cos(2*x + y) * sqrt(x*y)
y = linspace(0,1,N+1)
X,Y = np.meshgrid(x, y, indexing='ij')
F = F3(X,Y)
simp2dF = h**2/9 * w @ F @ w.T
print(f"simp2dF: {simp2dF}")

simpf: 1.022971259493227
errsimp: 0.0071072097854774086
simp2dF: -0.09026541105629891


In [6]:
from numpy import ones, linspace, loadtxt, pi, sqrt, cos, sin
from scipy.interpolate import CubicSpline
from scipy.fft import fft
from scipy.linalg import lstsq

tdata = [1,2,3,4,5,6,7,8,9]
ydata = [94.88,79.48,61.95,99.67,72.34,66.05,56.64,41.51,31.08]

teval = np.arange(10) + 0.5

spl = CubicSpline(tdata, ydata)
yspteval = spl(teval)
print(f"yspteval: {yspteval}")

poly = lambda t : np.array([1, t, t**2, sqrt(t), sin(pi*t/2)])

A = [poly(t) for t in tdata]
xLS = lstsq(A, ydata)[0]
print(f"xLS: {xLS}")

yLSteval = [poly(t) for t in teval] @ xLS
print(f"yLSteval: {yLSteval}")

m = 9
w = fft(ydata)
a0 = w[0].real/m

b = -2/m * w.imag[1:5]
print(f"b: {b}")

a = [-2.3688, 9.9369, 16.6633, 3.5819]
ydfteval = [a0 + np.sum([a[i] * cos(2*pi*(i+1)*t/m) + b[i] * sin(2*pi*(i+1)*t/m) for i in range(4)]) for t in teval]
print(f"ydfteval: {ydfteval}")

yspteval: [63.64970753 95.07255849 63.35494151 82.43767548 90.52935657 65.97364824
 62.98605048 49.04714984 35.14785016 30.4257492 ]
xLS: [ 171.44743728   60.70423126   -3.34496404 -141.0944602     6.16713255]
yLSteval: [105.55528355  86.53371981  74.85123919  74.61205131  81.93522856
  77.60052832  60.61768555  47.81103018  38.76306718  15.73311762]
b: [20.14594902  2.9660844   6.59141557  7.66351746]
ydfteval: [103.45877763268166, 55.83837394245037, 87.25902437128327, 88.6883618241127, 66.29096666666666, 62.45676281947209, 51.17281054320565, 29.080759390882974, 59.35416280924467, 103.45877763268167]


In [7]:
def do(N=5):
    h = 1/N
    x = linspace(0, 1, N+1)
    A = np.diag([-2] * (N-1)) + np.diag(np.ones(N-2), 1) + np.diag(np.ones(N-2), -1)
    p = lambda x : exp(x - 1)
    B = np.diag(p(x[1:-1])) @ (np.diag(np.ones(N-2), 1) - np.diag(np.ones(N-2), -1))
    sumB = B.sum()
    C = np.eye(N-1)
    R = np.zeros(N-1)
    R[0] = 1/h**2 - exp(h-1)/(2*h)
    Uapprox = sp.solve(-1/h**2 * A - 1/(2*h) * B + C, R)
    return Uapprox, A,sumB,C,R

Uapprox,A,sumB,C,R = do()
print(sumB)
print(f"Uapprox: {Uapprox}")

true = lambda x : (1-exp(1-x))/(1-exp(1))
x40 = linspace(0,1,41)
x80 = linspace(0,1,81)
true40 = true(x40[1:-1])
true80 = true(x80[1:-1])
E1 = norm(do(40)[0] - true40, np.inf)
E2 = norm(do(80)[0] - true80, np.inf)
E = [E1, E2]
print(f"E: {E}")


-0.36940178896076037
Uapprox: [0.71321487 0.47839554 0.28614985 0.12878524]
E: [1.2536427039799847e-06, 3.135926793795818e-07]
